In [1]:
import tensorflow as tf
import keras
import keras_nlp

EMBEDDING_DIM = 64
NUM_HEADS = 2
INTERMIDIATE_DIM = 256

dataset = [
    (
        "The quick brown fox jumps over the lazy dog.",
        "<start> A fox jumps over a dog.",
        "A fox jumps over a dog. <end>",
    ),
    (
        "An apple a day keeps the doctor away.",
        "<start> Eating an apple daily is healthy.",
        "Eating an apple daily is healthy. <end>",
    ),
]

# Preprocessing
input_texts, target_texts, decoder_target_text = zip(*dataset)

tokenizer = keras.preprocessing.text.Tokenizer(
    split=' ',
    filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
)
tokenizer.fit_on_texts(input_texts + target_texts + decoder_target_text)
vocab_size = len(tokenizer.word_index) + 1

input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)
decoder_target_sequences = tokenizer.texts_to_sequences(decoder_target_text)

max_input_length = max(len(seq) for seq in input_sequences)
max_target_length = max(len(seq) for seq in target_sequences)

input_sequences = keras.preprocessing.sequence.pad_sequences(
    input_sequences,
    maxlen=max_input_length,
    padding='post'
)
target_sequences = keras.preprocessing.sequence.pad_sequences(
    target_sequences,
    maxlen=max_target_length,
    padding='post'
)
decoder_target_sequences = keras.preprocessing.sequence.pad_sequences(
    decoder_target_sequences,
    maxlen=max_target_length,
    padding='post'
)
decoder_target_sequences = tf.expand_dims(decoder_target_sequences, axis=-1)

# Build model / Encoder & Decoder model
encoder_inputs = keras.Input(
    shape=(max_input_length,),
    name="encoder_inputs"
)
encoder_embedding = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=vocab_size,
    sequence_length=max_input_length,
    embedding_dim=EMBEDDING_DIM,
)(encoder_inputs)
encoder_outputs = keras_nlp.layers.TransformerEncoder(
    num_heads=NUM_HEADS,
    intermediate_dim=INTERMIDIATE_DIM,
)(encoder_embedding)

decoder_inputs = keras.Input(
    shape=(max_target_length,),
    name="decoder_inputs"
)
decoder_embedding = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=vocab_size,
    sequence_length=max_target_length,
    embedding_dim=EMBEDDING_DIM,
)(decoder_inputs)
decoder_outputs = keras_nlp.layers.TransformerDecoder(
    num_heads=NUM_HEADS,
    intermediate_dim=INTERMIDIATE_DIM,
)(decoder_embedding, encoder_outputs)

outputs = keras.layers.Dense(
    vocab_size,
    activation="softmax"
)(decoder_outputs)

model = keras.Model(
    [encoder_inputs, decoder_inputs],
    outputs
)
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Training
model.fit(
    [input_sequences, target_sequences],
    decoder_target_sequences,
    epochs=100,
)


Using TensorFlow backend


2024-06-04 09:59:32.459179: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-06-04 09:59:32.459196: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-06-04 09:59:32.459201: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-06-04 09:59:32.459226: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-04 09:59:32.459241: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2024-06-04 09:59:35.377956: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 3s 3s/step - loss: 3.3104 - accuracy: 0.0714
Epoch 2/100
1/1 [==============================] - 0s 43ms/step - loss: 2.3442 - accuracy: 0.2143
Epoch 3/100
1/1 [==============================] - 0s 41ms/step - loss: 1.6300 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 61ms/step - loss: 1.1213 - accuracy: 0.8571
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 0.7650 - accuracy: 0.8571
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 0.5239 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 0.3635 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2564 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1851 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1371 - accuracy: 1.0000
Epoch 11/100
1/1 [===============

In [2]:
def summarize(text):
    """
    Summarize text
    :param text: original text
    :return: summarized text
    """
    input_sequence = tokenizer.texts_to_sequences([text])
    input_sequence = keras.preprocessing.sequence.pad_sequences(
        input_sequence,
        maxlen=max_input_length,
        padding='post'
    )
    idx = tokenizer.word_index['<start>']
    decoder_input_sequence = tf.constant(
        [[idx]],
        dtype=tf.int64
    )
    
    summary = []
    for _ in range(max_target_length):
        predictions = model.predict(
            [input_sequence, decoder_input_sequence],
            verbose=0
        )
        next_token = tf.argmax(predictions[0, -1, :])
        next_word = tokenizer.index_word.get(next_token.numpy(), '<unk>')
        if next_word == '<end>':
            break
        summary.append(next_word)
        decoder_input_sequence = tf.concat(
            [decoder_input_sequence, tf.expand_dims([next_token], axis=-1)],
            axis=-1
        )
    return ' '.join(summary)

# Sample
sample_text = "The quick brown fox jumps over the lazy dog."
print("Original text:", sample_text)
print("Summary:", summarize(sample_text))

sample_text = "An apple a day keeps the doctor away."
print("Original text:", sample_text)
print("Summary:", summarize(sample_text))

sample_text = "A quick apple a day keeps the dog away."
print("Original text:", sample_text)
print("Summary:", summarize(sample_text))

Original text: The quick brown fox jumps over the lazy dog.
Summary: a fox jumps over a dog
Original text: An apple a day keeps the doctor away.
Summary: eating an apple daily is healthy
Original text: A quick apple a day keeps the dog away.
Summary: eating an apple daily is healthy


In [3]:
tokenizer.word_index

{'a': 1,
 'the': 2,
 'fox': 3,
 'jumps': 4,
 'over': 5,
 'dog': 6,
 'an': 7,
 'apple': 8,
 '<start>': 9,
 'eating': 10,
 'daily': 11,
 'is': 12,
 'healthy': 13,
 '<end>': 14,
 'quick': 15,
 'brown': 16,
 'lazy': 17,
 'day': 18,
 'keeps': 19,
 'doctor': 20,
 'away': 21}